# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("output_data/weather_output.csv", encoding="utf-8")
cities


,Cities,Countries,lat,lon,temp,humidity,cloudiness,wind speed,date
0,Mataura,NZ,-46.1927,168.8643,21.93,36,100,3.25,1645159626
1,Bengkulu,ID,-3.8004,102.2655,28.87,64,99,1.43,1645159627
2,Puerto Ayora,EC,-0.7393,-90.3518,23.87,89,99,2.95,1645159627
3,Effium,NG,6.6292,8.0593,25.17,80,40,1.22,1645159628
4,Keetmanshoop,NaN,-26.5833,18.1333,22.97,63,27,3.12,1645159629
...,...,...,...,...,...,...,...,...,...
559,Ola,RU,59.5833,151.2833,-15.80,76,79,5.25,1645160178
560,Rio de Janeiro,BR,-22.9028,-43.2075,23.00,94,75,1.54,1645160108
561,Wakkanai,JP,45.4094,141.6739,-4.01,74,75,9.26,1645160179
562,Zăvoi,RO,45.5167,22.4167,2.80,89,80,1.87,1645160179


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 # Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
humidity = cities["humidity"]
locations = cities[["lat", "lon"]]

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_df = cities.loc[(cities["temp"] >21.1) & (cities["temp"] <26.6) & (cities["wind speed"] <10) & (cities["cloudiness"] == 0)].dropna()
narrowed_df

,Cities,Countries,lat,lon,temp,humidity,cloudiness,wind speed,date
62,Kapaa,US,22.0752,-159.3190,22.97,78,0,6.69,1645159685
214,Kadhan,PK,24.4810,68.9859,25.58,24,0,4.17,1645159839
249,Ocós,GT,14.5094,-92.1933,24.94,69,0,3.89,1645159858
280,Vila Velha,BR,-20.3297,-40.2925,24.41,85,0,2.06,1645159896
345,Tura,IN,25.5198,90.2201,23.41,29,0,1.43,1645159952
423,Conde,BR,-7.2597,-34.9075,24.70,94,0,1.03,1645160038
447,Gwadar,PK,25.1216,62.3254,21.44,64,0,1.52,1645160051
460,Bela,IN,25.9333,81.9833,22.47,33,0,2.24,1645160080
505,Verāval,IN,20.9000,70.3667,26.06,34,0,3.81,1645160127
515,Pochutla,MX,15.7432,-96.4661,26.01,78,0,1.48,1645160133


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrowed_df
hotel_df["Hotel Name"] = ""
hotel_df

,Cities,Countries,lat,lon,temp,humidity,cloudiness,wind speed,date,Hotel Name
62,Kapaa,US,22.0752,-159.3190,22.97,78,0,6.69,1645159685,
214,Kadhan,PK,24.4810,68.9859,25.58,24,0,4.17,1645159839,
249,Ocós,GT,14.5094,-92.1933,24.94,69,0,3.89,1645159858,
280,Vila Velha,BR,-20.3297,-40.2925,24.41,85,0,2.06,1645159896,
345,Tura,IN,25.5198,90.2201,23.41,29,0,1.43,1645159952,
423,Conde,BR,-7.2597,-34.9075,24.70,94,0,1.03,1645160038,
447,Gwadar,PK,25.1216,62.3254,21.44,64,0,1.52,1645160051,
460,Bela,IN,25.9333,81.9833,22.47,33,0,2.24,1645160080,
505,Verāval,IN,20.9000,70.3667,26.06,34,0,3.81,1645160127,
515,Pochutla,MX,15.7432,-96.4661,26.01,78,0,1.48,1645160133,


In [8]:
params = {
    "radius": 8046.72,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat = row['lat']
    lon = row['lon']
    params['location'] = f'{lat},{lon}'
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
    except:
        print ('No hotel at', row['Cities'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
hotel_df

No hotel at Kadhan
No hotel at Ocós
No hotel at Conde
No hotel at Keti Bandar


,Cities,Countries,lat,lon,temp,humidity,cloudiness,wind speed,date,Hotel Name
62,Kapaa,US,22.0752,-159.3190,22.97,78,0,6.69,1645159685,The ISO
214,Kadhan,PK,24.4810,68.9859,25.58,24,0,4.17,1645159839,NA
249,Ocós,GT,14.5094,-92.1933,24.94,69,0,3.89,1645159858,NA
280,Vila Velha,BR,-20.3297,-40.2925,24.41,85,0,2.06,1645159896,Hotel Golden Tulip Porto Vitória
345,Tura,IN,25.5198,90.2201,23.41,29,0,1.43,1645159952,Hotel Polo Orchid
423,Conde,BR,-7.2597,-34.9075,24.70,94,0,1.03,1645160038,NA
447,Gwadar,PK,25.1216,62.3254,21.44,64,0,1.52,1645160051,Zaver Pearl Continental Hotel
460,Bela,IN,25.9333,81.9833,22.47,33,0,2.24,1645160080,Hotel Rajvilla
505,Verāval,IN,20.9000,70.3667,26.06,34,0,3.81,1645160127,Lords Inn Somnath
515,Pochutla,MX,15.7432,-96.4661,26.01,78,0,1.48,1645160133,Bahia de la Luna


In [11]:
print(hotel_df.columns)

Index(['Cities', 'Countries', 'lat', 'lon', 'temp', 'humidity', 'cloudiness',
       'wind speed', 'date', 'Hotel Name'],
      dtype='object')


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Countries}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))